## Mean-Reversion Strategy (using Bollinger Bands)

Importing dependencies

In [140]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

Fetching the data source and converting to pandas dataframe

In [141]:
data = pd.read_csv(filepath_or_buffer='../../resources/intraday.csv', parse_dates=['time'], index_col='time')

Getting details about the asset data by analysing the dataframe

In [142]:
data.head(10)

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
2018-01-03 04:00:00+00:00,1.203825
2018-01-03 10:00:00+00:00,1.202355
2018-01-03 16:00:00+00:00,1.201445
2018-01-03 22:00:00+00:00,1.201450


In [143]:
data.tail(10)

,Close
time,
2019-12-26 16:00:00+00:00,1.109655
2019-12-26 22:00:00+00:00,1.111890
2019-12-27 04:00:00+00:00,1.113860
2019-12-27 10:00:00+00:00,1.116300
2019-12-27 16:00:00+00:00,1.117580
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095
2019-12-30 16:00:00+00:00,1.119920


In [144]:
data.Close.to_frame()

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
...,...
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095


Getting more statistical insight into the dataset

In [145]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2070 entries, 2018-01-01 22:00:00+00:00 to 2019-12-30 22:00:00+00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2070 non-null   float64
dtypes: float64(1)
memory usage: 32.3 KB


In [146]:
data.describe()

,Close
count,2070.000000
mean,1.150400
std,0.041544
min,1.088635
25%,1.119714
50%,1.137698
75%,1.169170
max,1.253920


Since the only attribute we need is closing price
we modify the data frame


In [147]:
data = data.Close.to_frame()

In [148]:
data.head(10)

,Close
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
2018-01-03 04:00:00+00:00,1.203825
2018-01-03 10:00:00+00:00,1.202355
2018-01-03 16:00:00+00:00,1.201445
2018-01-03 22:00:00+00:00,1.201450


We plot the closing price of our asset

In [149]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [150]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

We proceed to define baseline and compute returns !needs improvement!

In [151]:
data['returns'] = np.log(data.div(data.shift(1)))

In [152]:
data

,Close,returns
time,,
2018-01-01 22:00:00+00:00,1.201205,NaN
2018-01-02 04:00:00+00:00,1.207055,0.004858
2018-01-02 10:00:00+00:00,1.204440,-0.002169
2018-01-02 16:00:00+00:00,1.205800,0.001129
2018-01-02 22:00:00+00:00,1.204690,-0.000921
...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092
2019-12-30 04:00:00+00:00,1.119940,0.000018
2019-12-30 10:00:00+00:00,1.120095,0.000138


Defining Mean-Reversion

In [153]:
SMA = 30
dev = 2

In [154]:
data['SMA'] = data['Close'].rolling(SMA).mean()

We try to visualise the asset price and SMA together

In [155]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))
fig.add_trace(go.Scatter(x=data.index, y=data.SMA, name=f'SMA - {SMA}'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

Viewing a shorter time interval for the asset

In [156]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].SMA, name=f'SMA - {SMA}'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

Plotting the standard deviation to assess risk of asset (! needs more elaboration on risk !)

In [157]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['Close'].rolling(SMA).std(), name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Volatility')

fig.show()

Since we want to incorporate bollinger bands we compute it

In [158]:
data['Lower'] = data['SMA'] - data['Close'].rolling(SMA).std() * dev
data['Upper'] = data['SMA'] + data['Close'].rolling(SMA).std() * dev

In [159]:
data

,Close,returns,SMA,Lower,Upper
time,,,,,
2018-01-01 22:00:00+00:00,1.201205,NaN,NaN,NaN,NaN
2018-01-02 04:00:00+00:00,1.207055,0.004858,NaN,NaN,NaN
2018-01-02 10:00:00+00:00,1.204440,-0.002169,NaN,NaN,NaN
2018-01-02 16:00:00+00:00,1.205800,0.001129,NaN,NaN,NaN
2018-01-02 22:00:00+00:00,1.204690,-0.000921,NaN,NaN,NaN
...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235


In [160]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))
fig.add_trace(go.Scatter(x=data.index, y=data.SMA, name=f'SMA - {SMA}'))
fig.add_trace(go.Scatter(x=data.index, y=data.Lower, name='Lower'))
fig.add_trace(go.Scatter(x=data.index, y=data.Upper, name='Upper'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [161]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].SMA, name=f'SMA - {SMA}'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Lower, name='Lower'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Upper, name='Upper'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

Dropping NaN values

In [162]:
data.dropna(inplace=True)

In [163]:
data

,Close,returns,SMA,Lower,Upper
time,,,,,
2018-01-11 04:00:00+00:00,1.194770,-0.000498,1.200504,1.190849,1.210160
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122
...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235


In [164]:
data['distance'] = data.Close - data.SMA

In [165]:
data['position'] = np.where(data.Close < data.Lower, 1, np.nan)
data['position'] = np.where(data.Close > data.Upper, -1, data['position'])

In [166]:
data

,Close,returns,SMA,Lower,Upper,distance,position
time,,,,,,,
2018-01-11 04:00:00+00:00,1.194770,-0.000498,1.200504,1.190849,1.210160,-0.005734,NaN
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,NaN
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,NaN
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,NaN
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0
...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0


In [167]:
data['position'] = np.where(data.distance * data.distance.shift(1) < 0, 0, data['position'])

In [168]:
data

,Close,returns,SMA,Lower,Upper,distance,position
time,,,,,,,
2018-01-11 04:00:00+00:00,1.194770,-0.000498,1.200504,1.190849,1.210160,-0.005734,NaN
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,0.0
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,NaN
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,NaN
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0
...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0


In [169]:
data.position = data.position.ffill().fillna(0)

In [170]:
data.position.value_counts()

 0.0    919
 1.0    596
-1.0    526
Name: position, dtype: int64

IMPROVE

In [171]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))
fig.add_trace(go.Scatter(x=data.index, y=data.SMA, name=f'SMA - {SMA}'))
fig.add_trace(go.Scatter(x=data.index, y=data.Lower, name='Lower'))
fig.add_trace(go.Scatter(x=data.index, y=data.Upper, name='Upper'))
fig.add_trace(go.Scatter(x=data.index, y=data.position, name='Position'), secondary_y=True)

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [172]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].SMA, name=f'SMA - {SMA}'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Lower, name='Lower'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Upper, name='Upper'))
fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].position, name='Position'), secondary_y=True)

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [173]:
data

,Close,returns,SMA,Lower,Upper,distance,position
time,,,,,,,
2018-01-11 04:00:00+00:00,1.194770,-0.000498,1.200504,1.190849,1.210160,-0.005734,0.0
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,0.0
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,0.0
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,0.0
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0
...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0


In [174]:
data['strategy'] = data.position.shift(1) * data['returns']

In [175]:
data.dropna(inplace=True)

In [176]:
data

,Close,returns,SMA,Lower,Upper,distance,position,strategy
time,,,,,,,,
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,0.0,0.000000
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,0.0,-0.000000
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,0.0,0.000000
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0,0.000000
2018-01-12 10:00:00+00:00,1.214820,0.001887,1.201086,1.189583,1.212589,0.013734,-1.0,-0.001887
...,...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0,-0.002092
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0,-0.000018
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0,-0.000138


In [177]:
data['creturns'] = data['returns'].cumsum().apply(np.exp)

In [178]:
data['cstrategy'] = data['strategy'].cumsum().apply(np.exp)

In [179]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=data.index, y=data.creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy, name='Returns (Strategy)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [180]:
data

,Close,returns,SMA,Lower,Upper,distance,position,strategy,creturns,cstrategy
time,,,,,,,,,,
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,0.0,0.000000,1.008491,1.000000
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,0.0,-0.000000,1.007060,1.000000
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,0.0,0.000000,1.008700,1.000000
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0,0.000000,1.014865,1.000000
2018-01-12 10:00:00+00:00,1.214820,0.001887,1.201086,1.189583,1.212589,0.013734,-1.0,-0.001887,1.016781,0.998115
...,...,...,...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0,-0.002092,0.937352,1.078555
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0,-0.000018,0.937369,1.078536
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0,-0.000138,0.937498,1.078387


Taking trading costs into consideration

In [181]:
ptc = 0.00007

In [182]:
data['trades'] = data.position.diff().fillna(0).abs()

In [183]:
data

,Close,returns,SMA,Lower,Upper,distance,position,strategy,creturns,cstrategy,trades
time,,,,,,,,,,,
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,0.0,0.000000,1.008491,1.000000,0.0
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,0.0,-0.000000,1.007060,1.000000,0.0
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,0.0,0.000000,1.008700,1.000000,0.0
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0,0.000000,1.014865,1.000000,1.0
2018-01-12 10:00:00+00:00,1.214820,0.001887,1.201086,1.189583,1.212589,0.013734,-1.0,-0.001887,1.016781,0.998115,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0,-0.002092,0.937352,1.078555,0.0
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0,-0.000018,0.937369,1.078536,0.0
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0,-0.000138,0.937498,1.078387,0.0


In [184]:
data.trades.value_counts()

0.0    1929
1.0     111
Name: trades, dtype: int64

In [185]:
data['strategy_net'] = data.strategy - data.trades * ptc

In [186]:
data['cstrategy_net'] = data.strategy_net.cumsum().apply(np.exp)

In [187]:
data

,Close,returns,SMA,Lower,Upper,distance,position,strategy,creturns,cstrategy,trades,strategy_net,cstrategy_net
time,,,,,,,,,,,,,
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,0.0,0.000000,1.008491,1.000000,0.0,0.000000,1.000000
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,0.0,-0.000000,1.007060,1.000000,0.0,-0.000000,1.000000
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,0.0,0.000000,1.008700,1.000000,0.0,0.000000,1.000000
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0,0.000000,1.014865,1.000000,1.0,-0.000070,0.999930
2018-01-12 10:00:00+00:00,1.214820,0.001887,1.201086,1.189583,1.212589,0.013734,-1.0,-0.001887,1.016781,0.998115,0.0,-0.001887,0.998045
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0,-0.002092,0.937352,1.078555,0.0,-0.002092,1.070207
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0,-0.000018,0.937369,1.078536,0.0,-0.000018,1.070188
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0,-0.000138,0.937498,1.078387,0.0,-0.000138,1.070040


In [188]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=data.loc['2018':].index, y=data.loc['2018':].creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.loc['2018':].index, y=data.loc['2018':].cstrategy, name='Returns (Strategy)'))
fig.add_trace(go.Scatter(x=data.loc['2018':].index, y=data.loc['2018':].cstrategy_net, name='Returns (Strategy + trading costs)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [189]:
data[['returns', 'strategy_net']].mean() * (4*252)

returns        -0.031776
strategy_net    0.033335
dtype: float64

In [190]:
data[['returns', 'strategy_net']].std() * np.sqrt(4*252)

returns         0.059975
strategy_net    0.044624
dtype: float64

Work on modifying the SMA bit